<h2>CatBoost ranking model</h2>
<p>Реализация модели ранжирования товаров с помощью библиотеки <code>CatBoost</code>.
Цель — предсказать топ-100 товаров для каждого пользователя.</p>

<h2>Импорт библиотек</h2>
<p>Подключаем Spark, функции для работы с данными, pandas для преобразований,
CatBoost для обучения модели ранжирования и scikit-learn для метрики NDCG, catboost для обучения модели ранжирования.
Инициализируем <code>findspark</code> для корректного запуска Spark из Python.</p>
<hr>

In [ ]:
from glob import glob
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, when, lit, unix_timestamp, countDistinct, avg, stddev, exp, explode, datediff, current_date, min as spark_min
from pyspark.sql import functions as F
import pandas as pd
from catboost import CatBoostRanker, Pool
from sklearn.metrics import ndcg_score
import numpy as np
from catboost import CatBoost
import os
findspark.init("/opt/spark")

<h2>Создание SparkSession</h2>
<p>Запускаем SparkSession с увеличенными лимитами памяти.
В логах оставляем только ошибки и включаем опцию игнорирования повреждённых файлов.</p>
<hr>

In [ ]:
spark = SparkSession.builder.appName("OzonApparelAnalysis") \
    .config("spark.driver.memory", "8g").config("spark.executor.memory", "8g").getOrCreate() # enabe big data
spark.sparkContext.setLogLevel("ERROR") # only errors logged
spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")

<h2>Загрузка parquet-файлов</h2>
<p>Собираем пути и читаем parquet-файлы:
<code>orders</code> (заказы), <code>tracker</code> (взаимодействия пользователей),
<code>items</code> (товары), <code>categories</code> (иерархия категорий),
<code>participants</code> (тестовая выборка).</p>
<hr>

In [ ]:
items_files = glob.glob('/srv/data/ml_ozon_recsys_train_final_apparel_items_data/*.parquet')
categories_tree = glob.glob('/srv/data/ml_ozon_recsys_train_final_categories_tree/*.parquet')
participants_files = glob.glob('/srv/data/ml_ozon_recsys_test_for_participants/*.parquet')
orders_files = glob.glob('/srv/data/preprocessed/orders_preprocessed/*.parquet')
tracker_files = glob.glob('/srv/data/preprocessed/tracker_preprocessed/*.parquet')
test_files = glob.glob('/srv/data/ml_ozon_recsys_test/*.parquet')

In [ ]:
orders = spark.read.parquet(*orders_files)
tracker = spark.read.parquet(*tracker_files)
items = spark.read.parquet(*items_files)
categories = spark.read.parquet(*categories_tree)

<h2>Формирование обучающего датасета</h2>
<p>Объединяем заказы, товары и пользовательские действия.
Добавляем количество взаимодействий <code>user_item_interactions</code>.
Целевая переменная <code>target</code>: 1 — заказ доставлен, иначе 0.</p>
<hr>

In [ ]:
train_data = (
    orders
    .join(items.select("item_id", "catalogid"), on="item_id", how="left")
    .join(tracker.groupBy("user_id", "item_id").count().withColumnRenamed("count", "user_item_interactions"),
          on=["user_id", "item_id"], how="left")
    .fillna(0, subset=["user_item_interactions"])
    .withColumn("target", F.when(F.col("last_status") == "delivered_orders", 1).otherwise(0))
)

<h2>Проверка распределения партиций</h2>
<p>Считаем количество строк в каждой партиции, чтобы проверить баланс нагрузки.</p>
<hr>

In [ ]:
partition_sizes = train_data.rdd.cache().mapPartitions(lambda iterator: [sum(1 for _ in iterator)]).collect()
print(partition_sizes)

<h2>Проверка распределения партиций</h2>
<p>Считаем количество строк в каждой партиции, чтобы проверить баланс нагрузки.</p>
<hr>

In [ ]:
train_data = train_data.repartition(80)

<h2>Проверка новых партиций</h2>
<p>Снова выводим распределение строк по партициям после перераспределения.</p>
<hr>

In [ ]:
partition_sizes = train_data.rdd.cache().mapPartitions(lambda iterator: [sum(1 for _ in iterator)]).collect()
print(partition_sizes)

In [ ]:
num = train_data.rdd.cache().getNumPartitions()

<h2>Обучение модели</h2>
<p>Обучаем модель CatBoostRanker по партициям. Для каждой партиции создаём <code>Pool</code>
с признаками и целевой переменной. Используется функция потерь <code>YetiRank</code>.
Сохраняем модель в файл <code>catboost_ranker_final.cbm</code>.</p>
<hr>

In [ ]:
prev_model = None

for idx in range(num):
    def keep_only_i(i):
        return lambda part_idx, it: it if part_idx == i else iter([])

    partition_df = train_data.rdd.mapPartitionsWithIndex(keep_only_i(idx)).toDF()
    train_pd = partition_df.toPandas().sort_values(["user_id"])

    train_pool = Pool(
        data=train_pd[["order_weight", "user_item_interactions"]],
        label=train_pd["target"],
        group_id=train_pd["user_id"]
    )

    model = CatBoostRanker(
        iterations=100,
        depth=6,
        learning_rate=0.1,
        loss_function="YetiRank",
        verbose=50
    )

    if prev_model is None:
        model.fit(train_pool)
    else:
        model.fit(train_pool, init_model=prev_model)

    train_preds = model.predict(train_pool)
    train_pd["score"] = train_preds

    prev_model = model
    print(f'{idx}/{num} batches processed')
    model.save_model("catboost_ranker_final.cbm")

<h2>Загрузка тестовых данных</h2>
<p>Читаем тестовую выборку из parquet-файла и сортируем по <code>user_id</code>.</p>
<hr>

In [ ]:
test_df = spark.read.parquet("/srv/data/preprocessed/final_test_only.parquet").orderBy('user_id')

<h2>Перепартиционирование теста</h2>
<p>Сохраняем тестовую выборку с разделением по <code>user_id</code> для дальнейшей обработки.</p>
<hr>

In [ ]:
test_df.write.mode("overwrite").partitionBy("user_id").parquet("repartitioned_test.parquet")

<h2>Чтение перепартиционированного теста</h2>
<p>Загружаем тестовую выборку с учётом новых партиций.</p>
<hr>

In [ ]:
test_df = spark.read.parquet("repartitioned_test.parquet")

<h2>Проверка распределения партиций теста</h2>
<p>Считаем количество строк в каждой партиции тестового датасета.</p>
<hr>

In [ ]:
partition_sizes = test_df.rdd.cache().mapPartitions(lambda iterator: [sum(1 for _ in iterator)]).collect()
print(partition_sizes)

<h2>Количество партиций в тесте</h2>
<p>Выводим число партиций в тестовой выборке.</p>
<hr>

In [ ]:
num = test_df.rdd.cache().getNumPartitions()

In [ ]:
num

<h2>Загрузка модели CatBoost</h2>
<p>Загружаем ранее обученную модель CatBoost из файла <code>catboost_ranker_final.cbm</code>.</p>
<hr>

In [ ]:
model = CatBoost()
model.load_model('catboost_ranker_final.cbm')

In [ ]:
columns = ['user_id'] + [f'item_id_{i}' for i in range(1, 101)]

In [ ]:
columns

<h2>Создание CSV для сабмита</h2>
<p>Открываем файл для записи результатов и задаём список колонок:
<code>user_id</code> и 100 предсказанных <code>item_id</code>.</p>
<hr>

In [ ]:
path = 'answer.csv'

In [ ]:
for idx in range(num):
    def keep_only_i(i):
        return lambda part_idx, it: it if part_idx == i else iter([])

    partition_df = test_df.rdd.cache().mapPartitionsWithIndex(keep_only_i(idx)).toDF()
    test_pd = partition_df.toPandas().sort_values(["user_id"])
    
    test_pool = Pool(
        data=test_pd[["order_weight", "user_item_interactions"]],
        group_id=test_pd["user_id"]
    )

    result = model.predict(test_pool)
    test_pd['rank'] = result
    rank = test_pd.sort_values(['user_id', 'rank'], ascending=[True, False]).groupby('user_id').head(100)
    top_items = rank.groupby('user_id')['item_id'].apply(lambda x: {f'item_id_{i}': val for i, val in enumerate(x)}).unstack(level=1).reset_index()
    if not os.path.exists(path):
        top_items.to_csv(path, mode='w', index=False)
    else:
        top_items.to_csv(path, mode='a', header=False, index=False)
    print(f'{idx}/{num} processed')

In [ ]:
def process_partition(partition):
    print(partition)

test_df.foreachPartition(process_partition)

In [ ]:
len(top_items[4368811])

In [ ]:
test_pool = Pool(
    data=test_pd[["order_weight", "user_item_interactions"]],
    group_id=test_pd["user_id"]
)

In [ ]:
test_preds = model.predict(test_pool)

In [ ]:
train_pd["score"] = train_preds

ndcg_list = []
for user, group in train_pd.groupby("user_id"):
    if len(group) < 2:
        continue
    y_true = group["target"].values.reshape(1, -1)
    y_score = group["score"].values.reshape(1, -1)
    ndcg = ndcg_score(y_true, y_score, k=100)
    ndcg_list.append(ndcg)